# Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup 
import pandas
from tqdm import tqdm

In [69]:
url = 'https://www.digikala.com/search/category-notebook-netbook-ultrabook/?pageno='

titles = []
engagements = []
stars = []
prices = []

for page in tqdm(range(1, 21)):
    
    '''' Enter your code here to Change the URL page '''
    new_url = url + str(page)
    
    page = requests.get(new_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Get name of product
    title = soup.select('div.c-product-box__title-en')
    for t in title:
        name = t.text
        titles.append(name)
        
    # Get price of product
    priceBoxes = soup.findAll('div', {'class': 'c-price__value c-price__value--plp js-plp-product-card-price'})
    for each_price in priceBoxes:
        # a PriceBox is containing initial price, discount %, and final price, we want only the final price
        price = each_price.find('div', {'class': 'c-price__value-wrapper'})
        price = price.text
        prices.append(price)

        
    ''' Enter your code here to get engagement number'''
    engagement = soup.select('span.c-product-box__engagement-rating-num')
    for t in engagement:
        name = t.text
        engagements.append(name)
    
    
    ''' Enter your code here to get star number'''
    
    star = soup.select('div.c-product-box__engagement-rating')
    for t in star:
        name = t.text
        stars.append(name)
    
    
    
# Saving info in a dictionary
product = {'Title': titles, 'Engagements': engagements, 'Stars': stars, 'Prices': prices}
# Saving dictionary in a dataframe
data = pandas.DataFrame.from_dict(product, orient='index')
data = data.transpose()
#data = data[data.Title != '']

100%|██████████| 20/20 [00:26<00:00,  1.31s/it]


<hr />

## Cleaning the table

<a name="Task2:clean"></a>


- Clean the pandas dataframe
 - Price, Star & Engagement should be Int or Float & None values must be replaced with 0
 
 
- Split Title into the 3 columns shown below & remove the original column
 - Size (inches): Float type
 - Brand: Object type
 - Model: Object type

### Put Zero for None Values in Columns

In [70]:
# Put Zero for None Values in Columns
def Zero(x):
    if x == None :
        x = 0
    return x
        
data['Engagements'] = data['Engagements'].apply(Zero) 
data['Stars'] = data['Stars'].apply(Zero) 
data['Prices'] = data['Prices'].apply(Zero) 



# For Engagement Column
import re
def For_Engagements(x):
    if x != 0 :
        x = re.findall('\\n.*\((\S.*)\).*' , str(x))[0]
    return int(x)
data['Engagements'] = data['Engagements'].apply(For_Engagements) 

# For Stars Column
def For_Stars(x):
    if x != 0 :
        x = re.findall('\\n *(\S.*)\\n.*\\n.*\\n.*' , str(x))[0]
    return float(x)
data['Stars'] = data['Stars'].apply(For_Stars) 


# For Price Column
def For_Prices(x):
    if x != 0 :
        x = re.findall('\\n *(\S*)\\n *' , str(x))[0]
#         x = re.sub('۰','0',str(x))
#         x = re.sub('۱','1',str(x))
#         x = re.sub('۲','2',str(x))
#         x = re.sub('۳','3',str(x))
#         x = re.sub('۴','4',str(x))
#         x = re.sub('۵','5',str(x))
#         x = re.sub('۶','6',str(x))
#         x = re.sub('۷','7',str(x))
#         x = re.sub('۸','8',str(x))
#         x = re.sub('۹','9',str(x))
        x = re.sub(',','',str(x))
    return float(x)/1000000


data['Prices'] = data['Prices'].apply(For_Prices)
data.rename(columns={"Prices":"Prices_Million_Toman"} , inplace=True)
data

,Title,Engagements,Stars,Prices_Million_Toman
0,ASUS VivoBook S533EQ - A 15.6 inch Laptop,158,4.2,38.400
1,Lenovo IdeaPad 3 15IGL05 - Z 15.6 inch Laptop,39,3.5,8.519
2,Lenovo Ideapad 1 - A - 11 inch Laptop,399,4.1,9.465
3,Apple MacBook Air MGN63 2020 - 13 inch Laptop,79,4.3,30.890
4,ASUS R565EA-BQ1366 15.6 inch Laptop,21,4.0,16.700
...,...,...,...,...
715,ASUS TUF GAMING FX506LI-HN118 15 inch Laptop,0,0.0,0.000
716,HP ZBook 15 Studio G5 Workstation-D2-15 Inch L...,0,0.0,0.000
717,Lenovo V15-NC 15.6 inch Laptop,0,0.0,0.000
718,Microsoft Surface Book 2- A - 15 inch Laptop,0,0.0,0.000


### Create Brand Column

In [71]:
def Brand(x):
    y = re.findall('^ *(\S*) ' , str(x))
    if y == []:
        return ' '
    else:
        return y[0]
           
data['Brand'] = data['Title'].apply(Brand)

#### Some of the Brands should be edit, for e.g , LENOVO and lenovo should Change into Lenovo & ...

In [72]:
def Change_Some_Brands_Name(x):
    if x == 'ACER':
        return 'Acer'
    if x == 'LENOVO' or x == 'lenovo' or x == 'LenovoThinkPad':
        return 'Lenovo'
    if x == 'Asus':
        return 'ASUS'
    if x == 'DELL' or x == 'Alienware':
        return 'Dell'
    if x == 'Pavilion':
        return 'HP'
    if x == 'Razer':
        return 'RazerBlade'
    else:
        return x
    
data['Brand'] = data['Brand'].apply(Change_Some_Brands_Name)
data = data[data.Brand != '14-CF2224NIA-B']

### Create Model Column

In [73]:
import re
def Model(x):
    y = re.findall('^\S* (.*) \d+\.?\d? ?[i I]nch [l L]aptop' , str(x))
    if y ==[]:
        y = re.findall('^\S* (.* i5) ?[i I]nch [l L]aptop' , str(x))
        if y == []:
            return x
        else:
            return y[0]
    if y!=[]:
        if y[0][-1]== '-':
            return y[0][:-2]
        else:
            return y[0]

data['Model'] = data['Title'].apply(Model)

<ipython-input-73-28e70c7b9aeb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Model'] = data['Title'].apply(Model)


### Create Size Column

In [74]:
def Inch(x):
    y = re.findall(' (\d+\.?\d?) ?[i I]nch' , str(x))
    if y ==[]:
        y = re.findall('i5 ?[i I]nch' , str(x))
        if y ==[]:
            y = re.findall('(\d+\.?\d?) ?[i I]nch' , str(x))
            if y ==[]:
                return ' '
            if y != []:
                return float(y[0])
            
        else:
            return 15
        
    else:
        return float(y[0])

data['Size_inch'] = data['Title'].apply(Inch)

<ipython-input-74-51238dfe872c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Size_inch'] = data['Title'].apply(Inch)


## Outlier data & incomplete data
 We could delete these data , because its a very **small** ratio of whole data and also they dont have:
* Title(Five rows first)
* Size(in inch)
* Price
* Stars
* Engagements

In [75]:
data[(data.Size_inch == ' ') | (data.Title == '')]

,Title,Engagements,Stars,Prices_Million_Toman,Brand,Model,Size_inch
43,,4,4.5,86.24,,,
77,,4,4.0,29.95,,,
280,,2,4.0,0.00,,,
307,,5,3.7,0.00,,,
547,,0,0.0,0.00,,,
589,Apple MacBook Pro With Retina Display-B,0,0.0,0.00,Apple,Apple MacBook Pro With Retina Display-B,
677,VAIO Pro 13 SVP13213SGS,0,0.0,0.00,VAIO,VAIO Pro 13 SVP13213SGS,
683,HP Mini 110-4132tu,0,0.0,0.00,HP,HP Mini 110-4132tu,
713,Samsung NP300E5V-S05AE,0,0.0,0.00,Samsung,Samsung NP300E5V-S05AE,


In [76]:
data = data[(data.Size_inch != ' ') & (data.Title != '')]

# Change Type of Size Column from string to Float
data['Size_inch'] = data['Size_inch'].astype(float)

### Drop Title Column & Reordering Columns

In [77]:
data = data.drop(["Title"],axis=1)
order_of_cols = ['Brand','Model','Size_inch','Engagements','Stars','Prices_Million_Toman']
data = data[order_of_cols]
data.head(15)

,Brand,Model,Size_inch,Engagements,Stars,Prices_Million_Toman
0,ASUS,VivoBook S533EQ - A,15.6,158,4.2,38.400
1,Lenovo,IdeaPad 3 15IGL05 - Z,15.6,39,3.5,8.519
2,Lenovo,Ideapad 1 - A,11.0,399,4.1,9.465
3,Apple,MacBook Air MGN63 2020,13.0,79,4.3,30.890
4,ASUS,R565EA-BQ1366,15.6,21,4.0,16.700
5,Microsoft,Surface Book 3- F,15.0,15,4.6,93.990
6,Lenovo,IdeaPad S145 - 15IGM,14.0,6,4.0,8.950
7,Apple,MacBook Pro MYD92 2020,13.0,47,4.4,43.990
8,Lenovo,Ideapad 3 15IIL05 - A,15.6,7,3.3,13.390
9,ASUS,ZenBook 13 UX325EA-KG287,13.3,53,4.4,32.990


<hr />

## Show all ASUS, 15.6 inch Laptops

In [80]:
data[(data.Brand == 'ASUS') & (data.Size_inch == 15.6) ]

,Brand,Model,Size_inch,Engagements,Stars,Prices_Million_Toman
0,ASUS,VivoBook S533EQ - A,15.6,158,4.2,38.40
4,ASUS,R565EA-BQ1366,15.6,21,4.0,16.70
16,ASUS,R521FA-EJ1071,15.6,5,3.8,16.40
40,ASUS,UX535LI-BO131T,15.6,104,4.5,43.00
47,ASUS,VivoBook R528EP-BQ253,15.6,3,3.7,34.45
...,...,...,...,...,...,...
691,ASUS,ROG Strix G15 G512LI-DH,15.6,0,0.0,0.00
700,ASUS,VivoBook K571GT-BQ934,15.6,0,0.0,0.00
705,ASUS,VivoBook R528EP-BQ240,15.6,0,0.0,0.00
707,ASUS,ZenBook 15 UX535LI-BN192,15.6,0,0.0,0.00



## Popular Brands
* Sort the Brands by popularity (Do not include 0 Enagement values)
* I suppose that poplarity relates on people's Comment

In [83]:
popular_brands = data[data.Engagements!=0].groupby(by='Brand').sum()
popular_brands.sort_values(by=['Engagements'] ,ascending=False)
popular_brands[['Engagements','Stars']]

,Engagements,Stars
Brand,,
ASUS,1384,263.2
Acer,492,84.4
Apple,379,61.2
Dell,629,185.5
HP,685,250.8
Huawei,113,27.2
Lenovo,5709,951.1
MSI,41,57.7
Microsoft,125,41.7


#### According to the table above, if we suppose the brand which got more Stars, is more popular , **LENOVO** is the most popular Brand
#### According to the table above, if we suppose the brand which got more Engagements, is more popular , **ASUS** is the most popular Brand
<hr />